# Exemplo de implementação de rede neural low-level.
Base de dados utilizada iris do lib sklearn.

Problema multi-classes utilizando a função de ativação SOFTMAX


In [0]:
# importação da lib sklearn importando os datasets
# repositório: archive.ics.uci.edu/ml/datasets/Iris
from sklearn import datasets
# vamos usar o exemplo iris()
iris = datasets.load_iris()
# atributos previsores (4 características)
X = iris.data
X

In [0]:
# atributos das classes
y = iris.target
y

In [0]:
# padronização dos dados
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
X

![alt text](https://drive.google.com/uc?id=1bCs3lSCjjfJet-cQhoRIvgvXshAE--Rz)

Como nosso exemplo tem 3 classes como resposta, precisamos de 3 neurônios de saída

In [0]:
# precisamos compatibilizar o numero de registros de saída com a quantidade de classes esperadas.
# para isso iremos utilizar o comando OneHotEncoder para que cada resposta tenha 3 atributos, que seja igual 
# a quantidade de neurônios na camada de saída.
from sklearn.preprocessing import OneHotEncoder
# passo como parametrao qual coluna quermos fazer a transformação.
# instancio um objeto.
onehot = OneHotEncoder()
y.shape

In [0]:
# transformando a resposta em formato de matriz
y = y.reshape(-1, 1)
y.shape

In [0]:
onehot.fit(y)
y = onehot.transform(y).toarray()
y

# agora temos as 3 classes representadas por codificação
classe 1 -> 1 0 0

classe 2 -> 0 1 0

classe 3 -> 0 0 1

In [0]:
# divisão da base em treinamento (70%) e teste (30%).
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [0]:
# conferindo a quantidade de regitros.
X_treinamento.shape

In [0]:
# conferindo a quantidade de regitros.
X_teste.shape

In [0]:
# importação do TensorFlow
%tensorflow_version 1.x
import tensorflow as tf
# importação do NumPy
import numpy as np
tf.__version__

In [0]:
# quantidade dos neurônios
# para a entrada o num. de neurônios é a quantidade de atributos.
neuronios_entrada = X.shape[1]
neuronios_entrada

In [0]:
# para a camada oculta
# podemos criar a quantidade baseado no calculo
# (quantidade de atributos previsores + quantidade de classes) / 2
# np.ceil arredonda para cima
neuronios_oculta = int(np.ceil((X.shape[1] + y.shape[1]) / 2))
neuronios_oculta

In [0]:
# neurônios na camada de saída que é igual ao num. de classes
neuronios_saida = y.shape[1]
neuronios_saida

![Modelo de RNA](https://drive.google.com/uc?id=1bCs3lSCjjfJet-cQhoRIvgvXshAE--Rz)

Modelo da RNA
<!-- ![Modelo da RNA](RNA2.png) -->

In [0]:
# criar os pesos, usamos uma estrutura de dados com keys
# para a camada oculta precisamos de uma matriz de 4 x 4 = 16 pesos.
# para a camada saída precisamos de uma matriz de 4 x 3 = 12 pesos.
# os valores do peso são criados com a função RANDOM.
W = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}

In [0]:
# pesos para a unidade de bias
# para a camada oculta teremos 4 pesos
# para a camada saída teremos 3 pesos.
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

![Modelo da RNA](https://drive.google.com/uc?id=1_-_xgpimyQ-R1TReFQrN0k7c3RjqDE28)



In [0]:
# criamos os places holders para X, com quantidade indefinida de linha e com 4 colunas
xph = tf.placeholder('float', [None, neuronios_entrada])
# criamos os places holders para Y, com quantidade indefinida de linha e com 3 colunas
yph = tf.placeholder('float', [None, neuronios_saida])

In [0]:
# criamos um função para facilitar na reutilização do códigos para testes.
# nesta função faremos os calculos do somatório dos peso, o calculo da função de ativação
# para as camadas da nossa rede.
def funRNA(x, W, bias):
    # somatório da camada oculta
    camada_oculta = tf.add(tf.matmul(x, W['oculta']), bias['oculta']) 
    # calculo func. ativação RELU
    camada_oculta_ativacao = tf.nn.relu(camada_oculta) 
    # somatório da camada de saída.
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, W['saida']), b['saida']) 
    # retorno dos valores da camada oculta.
    return camada_saida

In [0]:
# criamos um modelo que irá enviar como parametros para a func. os valores de pesos e as entradas da rede.
# esperamos como saída o calculo dos valores do somatório da camada de saída.
modelo = funRNA(xph, W, b)

In [0]:
# aqui faremos o calculo da função de ativação junto com o calculo do erro.
# estamos usando a função SOFTMAX como função de ativação neste modelo.
# para que a função faça o calculo do erro, precisamos passar como paramentros os valores
# que foram calculados pela rede (modelo) e os valores corretor (yph)
# modelo -> respostas que o modelo fez a previsão
# yph -> respostas corretas, vetor Y.
# func. reduce_mean faz a média do batch inteiro.
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
# vamos criar o otimizados, usando a função ADAM, que é similar ao gradiente de descida.
# passamos como parametro o erro, para que seja feita a otimização desse erro.
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [0]:
# tamanho da atualização dos registros.
batch_size = 8
batch_total = int(len(X_treinamento) / batch_size)
batch_total

In [0]:
# separa o vetor treinamento em 13 partes.
X_batches = np.array_split(X_treinamento, batch_total)
X_batches

In [0]:
# visualizando os valores do batch
X_batches[1] 

# Inicio do treinamento

In [0]:
# criamos uma sessão para inicializar o treinamento.
with tf.Session() as sess:
    # inicialização das variaveis.
    sess.run(tf.global_variables_initializer())
    # definimos o numero de épocas
    for epoca in range(3000):
        # variavel para calculo do erro.
        erro_medio = 0.0
        batch_total = int(len(X_treinamento) / batch_size)
        X_batches = np.array_split(X_treinamento, batch_total)
        # separamos batchs para o Y também
        y_batches = np.array_split(y_treinamento, batch_total)
        # calculamos e ajustamos os pesos para cada posição dos registros que separamos
        for i in range(batch_total):
            # pegamos os valores dos vetores x_batches e y_batches
            X_batch, y_batch = X_batches[i], y_batches[i]
            # passamos valores para os placeholders
            # rodamos nosso otimizador, e esperamos o valor do erro.            
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: X_batch, yph: y_batch})
            # calculo do erro médio
            erro_medio += custo / batch_total
        # exibimos os valores do erro para cada 500 registros.
        if epoca % 500 == 0:
            print('Época: ' + str((epoca + 1)) + ' erro: ' + str(erro_medio))
    # ao final do num. de épocas, temos os valores dos pesos ajustados.
    W_final, b_final = sess.run([W, b])

In [0]:
# visualizando o vetor de pesos.
W_final

In [0]:
# visualizando o vetor de bias.
b_final

# Previsões

com os valores de pesos ajustados, pordemo fazer algumas previsões.

In [0]:
# passamos para a func. previsões, os valores de pesos ajustados.
previsoes_teste = funRNA(xph, W_final, b_final)
with tf.Session() as sess:
    # inicializamos as variaveis.
    sess.run(tf.global_variables_initializer())
    # vamos executar a func. previsões, passando o vetor de teste como um placeholder.
    resp = sess.run(previsoes_teste, feed_dict = {xph: X_teste})
    # para o retorno da probabilidade de cada saída, excutamos a func. softmax.
    probabilidade = sess.run(tf.nn.softmax(resp))
    # para facilitar a visualização dos valores, vamos usar a func. argmax que retorna
    # o indice de onde esta o maior numero.
    resp_final = sess.run(tf.argmax(probabilidade, 1))

In [0]:
# visualizamos as previsões em percentual.
probabilidade

In [0]:
# visualizamos a resposta do maior percentual para cada resposta.
resp_final

# Avaliação da rede.

In [0]:
# para iniciar o calculo precisamos compatibilizar as respostas
# para isso utilizaremos a func. argmax no vetor de resultados também.
y_teste2 = np.argmax(y_teste, 1)
y_teste2

In [0]:
# para o calculo de precisão vamos usar a func. accuracy_score do sklearn.
from sklearn.metrics import accuracy_score
# calculo de precisão da rede
taxa_acerto = accuracy_score(y_teste2, resp_final)
# valor em %
taxa_acerto